#Lab 11 MNIST and Deep Learning CNN
#(https://wikidocs.net/60324 참고

In [1]:
import h5py
import pandas as pd
import numpy as np
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random
import torch.nn.init
import matplotlib.pyplot as plt
import os
import glob

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from tqdm.auto import tqdm

from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings(action='ignore') 

In [3]:
#reproducible한 실험을 하기 위해 시드를 고정해둘 필요가 있다. 
#Pytorch에서 무작위성을 배제하고 일관된 학습 결과를 얻으려면 다음과 같은 코드를 사용하면 된다. 
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

(https://moding.tistory.com/entry/Learning-Rate-Training-Epoch-Batch-Size%EC%9D%98-%EC%9D%98%EB%AF%B8)
learning_rate training epochs, batch_size 개념
1) learning_rate : alpha값, 기울기의 변화폭을 정해주는데 너무 커도, 너무작아도 안되므로 보통 lr=0.01로 시작한다.
2) 1 epoch : 전체 데이터셋에 대해 한 번 학습을 완료한 상태
3) batch_size : 한번 학습을 할때 데이터를 쪼갠 크기, 메모리의 한계와 속도 저하를 막기 위해
4) iteration : 1000개의 데이터셋, batch_size = 100이면 1epoch = 1000/batch_size = 10 iteration

### 뿐만아니라, transforms를 정의할 수 있고, shuffle, num_workers를 정의하는 등 다양한 option 값으로 매우 손 쉽게 datasets를 컨트롤 할 수 있습니다.

In [4]:
CFG = {
    'EPOCHS':15,
    'LEARNING_RATE':1e-3,
    'BATCH_SIZE':16,
    'SEED':41
}

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [29]:
all_df = pd.read_csv('C:/Users/sarah/data/open/train.csv')
all_points = h5py.File('C:/Users/sarah/data/open/train.h5', 'r')

all_points

<HDF5 file "train.h5" (mode r)>

In [7]:
train_df = all_df.iloc[:int(len(all_df)*0.8)]
val_df = all_df.iloc[int(len(all_df)*0.8):]

train_df.head()

,ID,label
0,0,5
1,1,0
2,2,4
3,3,1
4,4,9


In [ ]:
import numpy as np
from PIL import Image

img_name = ('train.h5')

In [8]:
class CustomDataset(Dataset):
    def __init__(self, id_list, label_list, point_list):
        self.id_list = id_list
        self.label_list = label_list
        self.point_list = point_list
        
    def __getitem__(self, index):
        image_id = self.id_list[index]
        
        # h5파일을 바로 접근하여 사용하면 학습 속도가 병목 현상으로 많이 느릴 수 있습니다.
        points = self.point_list[str(image_id)][:]
        image = self.get_vector(points)
        
        if self.label_list is not None:
            label = self.label_list[index]
            return torch.Tensor(image).unsqueeze(0), label
        else:
            return torch.Tensor(image).unsqueeze(0)
    
    def get_vector(self, points, x_y_z=[16, 16, 16]):
        # 3D Points -> [16,16,16]
        xyzmin = np.min(points, axis=0) - 0.001
        xyzmax = np.max(points, axis=0) + 0.001

        diff = max(xyzmax-xyzmin) - (xyzmax-xyzmin)
        xyzmin = xyzmin - diff / 2
        xyzmax = xyzmax + diff / 2

        segments = []
        shape = []

        for i in range(3):
            # note the +1 in num 
            if type(x_y_z[i]) is not int:
                raise TypeError("x_y_z[{}] must be int".format(i))
            s, step = np.linspace(xyzmin[i], xyzmax[i], num=(x_y_z[i] + 1), retstep=True)
            segments.append(s)
            shape.append(step)

        n_voxels = x_y_z[0] * x_y_z[1] * x_y_z[2]
        n_x = x_y_z[0]
        n_y = x_y_z[1]
        n_z = x_y_z[2]

        structure = np.zeros((len(points), 4), dtype=int)
        structure[:,0] = np.searchsorted(segments[0], points[:,0]) - 1
        structure[:,1] = np.searchsorted(segments[1], points[:,1]) - 1
        structure[:,2] = np.searchsorted(segments[2], points[:,2]) - 1

        # i = ((y * n_x) + x) + (z * (n_x * n_y))
        structure[:,3] = ((structure[:,1] * n_x) + structure[:,0]) + (structure[:,2] * (n_x * n_y)) 

        vector = np.zeros(n_voxels)
        count = np.bincount(structure[:,3])
        vector[:len(count)] = count

        vector = vector.reshape(n_z, n_y, n_x)
        return vector

    def __len__(self):
        return len(self.id_list)

In [9]:
# dataset loader
train_dataset = CustomDataset(train_df['ID'].values, train_df['label'].values, all_points)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_df['ID'].values, val_df['label'].values, all_points)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

### drop_last = True
drop_last를 하는 이유를 이해하기 위해서 1,000개의 데이터가 있다고 했을 때, 배치 크기가 128이라고 해봅시다. 1,000을 128로 나누면 총 7개가 나오고 나머지로 104개가 남습니다. 이때 104개를 마지막 배치로 한다고 하였을 때 128개를 충족하지 못하였으므로 104개를 그냥 버릴 수도 있습니다. 이때 마지막 배치를 버리려면 drop_last=True를 해주면 됩니다. 이는 다른 미니 배치보다 개수가 적은 마지막 배치를 경사 하강법에 사용하여 마지막 배치가 상대적으로 과대 평가되는 현상을 막아줍니다.

In [25]:
#CNN MODEL
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        self.layer1 = nn.Sequential(
            nn.Conv3d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm3d(32),
            torch.nn.ReLU(),
            nn.MaxPool3d(kernel_size=2, stride=2))
        self.layer2 = torch.nn.Sequential(
            nn.Conv3d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm3d(64),
            torch.nn.ReLU(),
            nn.MaxPool3d(kernel_size=2, stride=2))
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv3d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm3d(128),
            torch.nn.ReLU(),
            nn.MaxPool3d(kernel_size=2, stride=2, padding=1))
        #-------------------------------------------------------------#
        self.fc1 = nn.Linear(3*3*3 * 128, 625, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight) #가중치 초기화 : xavier_uniform or HE_uniform 방법이 주로 쓰이는데 he_uniform이 최근 자주쓰임
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            nn.Dropout(p=1 - self.keep_prob))
        
        # L5 Final FC 625 inputs -> 10 outputs
        self.fc2 = nn.Linear(625, 10, bias=True)
        nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.layer4(out)
        out = self.fc2(out)
        return out   

In [26]:
# instantiate CNN model
model = CNN().to(device)

In [27]:
# define cost/loss & optimizer
#cross entrophy : info가 틀릴수 있는 정도(확률), softmax(Q라고 쓰임) : Estimated Probability, P는 True Probability이다.

# torch.nn.CrossEntropyLoss는 nn.LogSoftmax와 nn.NLLLoss의 연산의 조합이다.
#nn.LogSoftmax는 신경망 말단의 결과 값들을 확률개념으로 해석하기 위한 Softmax 함수의 결과에 log 값을 취한 연산이고,
#nn.NLLLoss는 nn.LogSoftmax의 log 결과값에 대한 교차 엔트로피 손실 연산(Cross Entropy Loss|Error)
criterion = torch.nn.CrossEntropyLoss()    # Softmax is internally computed. 
optimizer = torch.optim.Adam(model.parameters(), lr=CFG['LEARNING_RATE'])
#머신러닝 알고리즘에서 최적화는 비용함수의 값이 가장 작아지는 최적의 파라미터를 찾는 과정을 말하며, 비용함수는 손실함수를 사용하여 정의될 수 있다.

In [28]:
# # train my model
# total_batch = len(train_loader)
# model.train()    # set the model to train mode (dropout=True)
# print('Learning started. Waiting for training')
# for epoch in range(CFG['EPOCHS']):
#     avg_cost = 0

#     for X, Y in train_loader:
#         X = X.to(device)
#         Y = Y.to(device)

#         optimizer.zero_grad()
#         hypothesis = model(X)
#         cost = criterion(hypothesis, Y)
#         cost.backward()
#         optimizer.step()

#         avg_cost += cost / total_batch

#     print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

# print('Learning Finished!')

Learning started. Waiting for training
[Epoch:    1] cost = 0.382512152
[Epoch:    2] cost = 0.152759239
[Epoch:    3] cost = 0.120004125
[Epoch:    4] cost = 0.0873160064
[Epoch:    5] cost = 0.0743927658
[Epoch:    6] cost = 0.0630536675
[Epoch:    7] cost = 0.0509901792
[Epoch:    8] cost = 0.0424067266
[Epoch:    9] cost = 0.0366119742
[Epoch:   10] cost = 0.0303025246
[Epoch:   11] cost = 0.0296249781
[Epoch:   12] cost = 0.0262725595
[Epoch:   13] cost = 0.0228511039
[Epoch:   14] cost = 0.0186515767
[Epoch:   15] cost = 0.0200849827
Learning Finished!


In [35]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    best_score = 0
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for data, label in tqdm(iter(train_loader)):
            data, label = data.float().to(device), label.long().to(device)
            optimizer.zero_grad()
            
            output = model(data)
            loss = criterion(output, label)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
        
        if scheduler is not None:
            scheduler.step()
            
        val_loss, val_acc = validation(model, criterion, val_loader, device)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss)}] Val Loss : [{val_loss}] Val ACC : [{val_acc}]')
        
        if best_score < val_acc:
            best_score = val_acc
            torch.save(model.state_dict(), './best_model.pth')

In [36]:
def validation(model, criterion, val_loader, device):
    model.eval()
    true_labels = []
    model_preds = []
    val_loss = []
    with torch.no_grad():
        for data, label in tqdm(iter(val_loader)):
            data, label = data.float().to(device), label.long().to(device)
            
            model_pred = model(data)
            loss = criterion(model_pred, label)
            
            val_loss.append(loss.item())
            
            model_preds += model_pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += label.detach().cpu().numpy().tolist()
    
    return np.mean(val_loss), accuracy_score(true_labels, model_preds)

In [38]:
model = CNN()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = None

train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch : [1] Train Loss : [0.38435008316840974] Val Loss : [0.097418234183616] Val ACC : [0.9707]


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

...

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch : [14] Train Loss : [0.020003415417765406] Val Loss : [0.09595837879376817] Val ACC : [0.9842]


  0%|          | 0/2500 [00:00<?, ?it/s]

  0%|          | 0/625 [00:00<?, ?it/s]

Epoch : [15] Train Loss : [0.021011208806646038] Val Loss : [0.10773235138708169] Val ACC : [0.984]


## Inference

In [39]:
test_df = pd.read_csv('C:/Users/sarah/data/open/sample_submission.csv')
test_points = h5py.File('C:/Users/sarah/data/open/test.h5', 'r')

In [40]:
test_dataset = CustomDataset(test_df['ID'].values, None, test_points)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [42]:
checkpoint = torch.load('./best_model.pth')
model = CNN()
model.load_state_dict(checkpoint)
model.eval()

CNN(
  (layer1): Sequential(
    (0): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (1): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool3d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=3456, out_features=625, bias=True)
  (layer4): Sequential(
    

In [43]:
def predict(model, test_loader, device):
    model.to(device)
    model.eval()
    model_preds = []
    with torch.no_grad():
        for data in tqdm(iter(test_loader)):
            data = data.float().to(device)
            
            batch_pred = model(data)
            
            model_preds += batch_pred.argmax(1).detach().cpu().numpy().tolist()
    
    return model_preds

In [44]:
preds = predict(model, test_loader, device)

  0%|          | 0/2500 [00:00<?, ?it/s]

## Submit

In [46]:
test_df['label'] = preds
test_df.to_csv('C:/Users/sarah/data/open/submit.csv', index=False)